<center><h1 style="font-size:40px;">Exercise II: Regression</h1></center>

---

Now we are going to look at a regression problem. The data as described above (regr1) consists of 6 inputs (features) and one output (target) value. As for previous examples a new data set is generated each time you call the *regr1* function. To get exactly the same data set between different calls, use a fixed seed. New for this problem is that one can also control the amount of noise added to the target value. We are going to use a relatively small training dataset (\~250) and a larger validation dataset (\~1000) to get a more robust estimation of the generalization performance. For regression problems we also need new performance measures. The *stats_reg* function will give you two such measures:
* MSE = mean squared error (low error mean good performance)
* CorrCoeff = Pearson correlation coefficient for the scatter plot between predicted and true values.

# Data 
## regr1
There is also a synthetic regression problem, called *regr1*. It has 6 inputs (independent variables) and one output variable (dependent variable). It is generated according to the following formula:  

$\qquad d = 2x_1 + x_2x_3^2 + e^{x_4} + 5x_5x_6 + 3\sin(2\pi x_6) + \alpha\epsilon$  
    
where $\epsilon$ is added normally distributed noise and $\alpha$ is a parameter controlling the size of the added noise. Variables $x_1,...,x_4$ are normally distrubuted with zero mean and unit variance, whereas $x_5, x_6$ are uniformly distributed ($[0,1]$). The target value $d$ has a non-linear dependence on ***x***.


# Tasks

## Task 1
Use 250 data points for training and about 1000 for validation and **no** added noise. Train an MLP to predict the target output. If you increase the complexity of the model (e.g. number of hidden nodes) you should be able to reach a very small training error. You will also most likely see that the validation error decreases as you increase the complexity or at least no clear sign of overtraining. 

**Note:** As with previous examples you may need to tune the optimization parameters to make sure that you have "optimal" training. That is, increase or decrease the learningrate, possibly train longer times (increase *epochs*) and change the *batch_size* parameter.

**TODO:** Even though the validation error is most likely still larger than the training error why do we not see any overtraining of the model? (Hint: What is it that typically causes overfitting?)

## Task 2
Use the same training and validation data sets as above, but add 0.4 units of noise (set the second parameter when calling the *regr1* function to 0.4 for both training and validation). Now train again, starting with a "small" model and increase the number of hidden nodes as you monitor the validation result for each model. Make a note of the validation error you obtained a this point!

**TODO:** How many nodes do you have for opitimal validation performance, i.e. more hidden nodes results in overtraining?

## Task 3
Instead of using the number of hidden nodes to control the complexity it is often better to use a regularization term added to the error function. You are now going to control the complexity by adding a *L2* regularizer (see the "INPUT" dictionary in the cell). You should modify this value until you find the "near optimal" validation performance. Use 15 hidden nodes. 

**TODO:** Give the L2 value that you found to give "optimal" validation performance and compare with the result from  question 7 (optimal performance).

## Task 4
**TODO:** Summarize your findings in a few sentences.


# Code

The following code allows us to edit imported files without restarting the kernel for the notebook

In [1]:
%load_ext autoreload
%autoreload 2

# Hacky solution to access the global utils package
import sys,os
sys.path.append(os.path.dirname(os.path.realpath('')))

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pytorch_lightning as pl

from config import LabConfig
from dataset import MLPData
from utils.model import Model
from utils.progressbar import LitProgressBar
from utils.model import Model
from torch.utils.data import TensorDataset, DataLoader
from utils import (
    plot,
    progressbar
) 

In [3]:
cfg = LabConfig()

# Defining the MLP model

This cell defines the MLP model. There are a number of parameters that is needed to 
define a model. Here is a list of them: **Note:** They can all be specified when you call
this function in later cells. The ones specified in this cell are the default values.


In [4]:
class MLP(torch.nn.Module):
    def __init__(self, 
                inp_dim=None,         
                hidden_nodes=1, # number of nodes in hidden layer
                num_out=None,
                **kwargs
            ):
        super(MLP, self).__init__()
        self.fc1 = torch.nn.Linear(inp_dim, hidden_nodes)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_nodes, num_out)


    def forward(self, x):
        hidden = self.fc1(x)
        relu = self.relu(hidden)
        output = self.fc2(relu)
        return torch.tanh(output)

## Define a function that allow us to convert numpy to pytorch DataLoader

In [5]:
def numpy2Dataloader(x,y, batch_size=25, num_workers=10,**kwargs):
    return DataLoader(
        TensorDataset(
            torch.from_numpy(x).float(), 
            torch.from_numpy(y).unsqueeze(1).float()
        ),
        batch_size=batch_size,
        num_workers=num_workers,
        **kwargs
    )

In [6]:
# Generate training and validation data
x_train, d_train = MLPData.regr1(250, 0.0) # 250 data points with no noise
x_val, d_val = MLPData.regr1(1000, 0.0)

# Here we need to normalize the target values
norm_m = d_train.mean(axis=0)
norm_s = d_train.std(axis=0)
d_train = (d_train - norm_m) / norm_s

# We use the same normalization for the validation data.
d_val = (d_val - norm_m) / norm_s

train_loader = numpy2Dataloader(x_train,d_train)
val_loader =  numpy2Dataloader(x_val,d_val)

## Configuration
Setup our local config that should be used for the trainer.

In [7]:
config = {
    'max_epochs':20,
    'model_params':{
        'inp_dim':x_train.shape[1],         
        'hidden_nodes':1,   # activation functions for the hidden layer
        'num_out':1 # if binary --> 1 |  regression--> num inputs | multi-class--> num of classes
    },
    'criterion':torch.nn.MSELoss(), # error function
    'optimizer':{
        "type":torch.optim.Adam,
        "args":{
            "lr":0.005,
        }
    }
}

## Training

Lastly, put everything together and call on the trainers fit method. 

In [8]:
model = Model(MLP(**config["model_params"]),**config)

trainer = pl.Trainer(
            max_epochs=config['max_epochs'], 
            gpus=cfg.GPU,
            logger=pl.loggers.TensorBoardLogger(save_dir=cfg.TENSORBORD_DIR),
            callbacks=[LitProgressBar()],
            progress_bar_refresh_rate=1,
            weights_summary=None, # Can be None, top or full
            num_sanity_val_steps=10,   
        )
trainer.fit(
    model, 
    train_dataloader=train_loader,
    val_dataloaders=val_loader
);

/home/stud/d/denmar20/.conda/envs/deeplearn/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:90: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=1)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
/home/stud/d/denmar20/.conda/envs/deeplearn/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:167: LightningDeprecationWarning: Setting `Trainer(weights_summary=None)` is deprecated in v1.5 and will be removed in v1.7. Please set `Trainer(enable_model_summary=False)` instead.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/stud/d/denmar20/.conda/envs/dee

/home/stud/d/denmar20/.conda/envs/deeplearn/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1805: LightningDeprecationWarning: `trainer.progress_bar_dict` is deprecated in v1.5 and will be removed in v1.7. Use `ProgressBarBase.get_metrics` instead.
  rank_zero_deprecation(


## Evaluation

In [9]:
plot.stats_class(x_train, d_train, 'Training', model)
plot.stats_class(x_val, d_val, 'Validation', model)

ValueError: No False Positive cant be zero

In [10]:
d_train

array([ 0.58561936,  0.98602412,  0.36361025,  2.06383404,  0.06505802,
       -0.82929667,  0.08070941,  0.19945641,  0.53680017,  0.57401835,
        0.15227812,  0.69215745,  0.23303388, -1.06518951, -0.64961266,
       -1.48501043, -1.15671963,  0.82317982, -0.32526958,  0.61388274,
        1.57067767,  0.48804659, -1.08018679,  0.92410638, -0.3922601 ,
       -1.11296605,  2.15298778,  2.49156797,  1.68488207,  0.35771921,
        0.28726033,  0.7799129 ,  0.67479483, -0.76973385,  0.31020062,
       -0.98434003,  2.34804886,  0.08251796,  0.93566323, -0.38817927,
        0.75242135,  0.23810264, -0.14020385,  0.27518528, -1.40467257,
        1.75371091,  1.98188934,  0.55465896,  0.36868815,  0.23555466,
       -0.61289734, -1.38042114,  0.01740044, -0.35260033, -1.52687007,
       -1.35077443,  0.06172444,  0.60179745, -1.69715578, -0.05016608,
        0.92625793, -1.2223584 ,  0.0518899 ,  0.42471049, -1.179242  ,
       -0.5978531 , -1.15384908, -1.49230972,  0.38789304, -0.01

In [15]:
def model_eval(config_, train_loader_, val_loader_):
    model = Model(MLP(**config_["model_params"]),**config_)

    trainer = pl.Trainer(
                max_epochs=config['max_epochs'], 
                gpus=cfg.GPU,
                logger=pl.loggers.TensorBoardLogger(save_dir=cfg.TENSORBORD_DIR),
                callbacks=[LitProgressBar()],
                progress_bar_refresh_rate=1,
                weights_summary=None, # Can be None, top or full
                num_sanity_val_steps=10,   
            )
    trainer.fit(
        model, 
        train_dataloader=train_loader_,
        val_dataloaders=val_loader_
    );

In [16]:
# Task 1 copy of code with changes

config = {
    'max_epochs':150,
    'model_params':{
        'inp_dim':x_train.shape[1],         
        'hidden_nodes':400,   # activation functions for the hidden layer
        'num_out':1 # if binary --> 1 |  regression--> num inputs | multi-class--> num of classes
    },
    'criterion':torch.nn.MSELoss(), # error function
    'optimizer':{
        "type":torch.optim.Adam,
        "args":{
            "lr":0.01,
        }
    }
}

model_eval(config, train_loader, val_loader)

/home/stud/d/denmar20/.conda/envs/deeplearn/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:90: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=1)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
/home/stud/d/denmar20/.conda/envs/deeplearn/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:167: LightningDeprecationWarning: Setting `Trainer(weights_summary=None)` is deprecated in v1.5 and will be removed in v1.7. Please set `Trainer(enable_model_summary=False)` instead.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/stud/d/denmar20/.conda/envs/dee

In [17]:
x_train, d_train = MLPData.regr1(250, 0.4) # 250 data points with no noise
x_val, d_val = MLPData.regr1(1000, 0.4)

# Here we need to normalize the target values
norm_m = d_train.mean(axis=0)
norm_s = d_train.std(axis=0)
d_train = (d_train - norm_m) / norm_s

# We use the same normalization for the validation data.
d_val = (d_val - norm_m) / norm_s

train_loader = numpy2Dataloader(x_train,d_train)
val_loader =  numpy2Dataloader(x_val,d_val)

for hidden_nodes in [1, 10, 20, 50, 100, 400, 1000]:
    print('hidden_nodes: ' + str(hidden_nodes))
    config = {
        'max_epochs':150,
        'model_params':{
            'inp_dim':x_train.shape[1],         
            'hidden_nodes':hidden_nodes,   # activation functions for the hidden layer
            'num_out':1 # if binary --> 1 |  regression--> num inputs | multi-class--> num of classes
        },
        'criterion':torch.nn.MSELoss(), # error function
        'optimizer':{
            "type":torch.optim.Adam,
            "args":{
                "lr":0.01,
            }
        }
    }
    model_eval(config, train_loader, val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


hidden_nodes: 1


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


hidden_nodes: 10


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


...

hidden_nodes: 100


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


hidden_nodes: 400


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


hidden_nodes: 1000


In [18]:
# Task 3

for l2 in [0.001, 0.005, 0.01, 0.05, 0.09]:
    print('hidden_nodes: ' + str(hidden_nodes))
    config = {
    'max_epochs':100,
    'model_params':{
        'inp_dim':x_train.shape[1],         
        'hidden_nodes':15,   # activation functions for the hidden layer
        'num_out':x_train.shape[0] # if binary --> 1 |  regression--> num inputs | multi-class--> num of classes
    },
    'criterion':torch.nn.MSELoss(), # error function
    'optimizer':{
        "type":torch.optim.Adam,
        "args":{
            "lr":0.015,
            'weight_decay': l2,
        }
    }
}
    model_eval(config, train_loader, val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


hidden_nodes: 1000


/home/stud/d/denmar20/.conda/envs/deeplearn/lib/python3.9/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([25, 1])) that is different to the input size (torch.Size([25, 250])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


hidden_nodes: 1000{'loss': '0.444'}


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


...

hidden_nodes: 1000
hidden_nodes: 1000{'loss': '0.57'}


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


GPU available: False, used: False


hidden_nodes: 1000


TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
